In [1]:
import pandas as pd
import requests
import json


In [2]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [6]:
arrest_data = 'https://data.cityofnewyork.us/resource/uip8-fykc.json'
response_arrest_data = requests.get(arrest_data).json()
print(json.dumps(response_arrest_data, indent=4))

[
    {
        "arrest_key": "280255493",
        "arrest_date": "2024-01-10T00:00:00.000",
        "pd_cd": "397",
        "pd_desc": "ROBBERY,OPEN AREA UNCLASSIFIED",
        "ky_cd": "105",
        "ofns_desc": "ROBBERY",
        "law_code": "PL 1601001",
        "law_cat_cd": "F",
        "arrest_boro": "M",
        "arrest_precinct": "26",
        "jurisdiction_code": "0",
        "age_group": "<18",
        "perp_sex": "M",
        "perp_race": "BLACK",
        "x_coord_cd": "996342",
        "y_coord_cd": "236149",
        "latitude": "40.814845",
        "longitude": "-73.956312",
        "geocoded_column": {
            "type": "Point",
            "coordinates": [
                -73.956312,
                40.814845
            ]
        },
        ":@computed_region_f5dn_yrer": "37",
        ":@computed_region_yeji_bk3q": "4",
        ":@computed_region_92fq_4b7q": "23",
        ":@computed_region_sbqj_enih": "17",
        ":@computed_region_efsh_h5xi": "12424"
    },
    